# Team SharkiShark

### TODO

##### The Bot Itself

- enums for direction, player color and field state (obstructed, player, empty)
- validate moves
- pick random valid move
- fitness/scoring function - how well are we doing / good is our current situation
    - how much greater is our current swarm
    - how many swarms vs. how many rounds left

##### The Communication with the Server

- client sends command
- async message handling
    1. thread receives command
    2. parses it into GameState
    3. passes it async to GameLogic, which inturn sends back the response (async)


In [2]:
import socket

HOST = 'localhost'
PORT = 13052

## Experimenting

In [3]:
def receive(sock):
    data = ""
    try:
        while 1:
            b = sock.recv(1)

            if not b:
                print(b)
                break

            data += b.decode('utf8')
    except socket.timeout:
        pass
    return data

In [4]:
import threading

class StoppableThread(threading.Thread):
    '''
        Thread class with a stop() method. The thread itself has to check regularly for the stopped() condition.
    '''

    def __init__(self):
        super(StoppableThread, self).__init__()
        self._stop_event = threading.Event()

    def stop(self):
        self._stop_event.set()

    def is_stopped(self):
        return self._stop_event.is_set()

    
    
    
class GameLogic():
    
    def onStatusUpdate(gamefield):
        print('-> onStatusUpdate()')
        
    def onMoveRequest():
        print('-> onMoveRequest()')


class Client(StoppableThread):

    def __init__(self, host, port, gameLogic):
        super().__init__()
        self.gameLogic = gameLogic
        
        self.sock = socket.socket()
        self.sock.connect((host, port))
    
    def run(self):
        self.sock.settimeout(0.5)
        with self.sock:
            while not self.is_stopped():
                try:
                    message = receive(self.sock)
                    self.parseMessage(message)
                except socket.error:
                    pass
                except Exception as ex:
                    print(ex)
                    raise ex
    
    def parseMessage(self, message):
        print('-> parseMessage(%s)' % message)
    
    def send(self, message):
        self.sock.send(message.encode('utf8'))

    def move(self, roomId, posX, posY, directionString):

        #let hints = move.debugHints.reduce(into: "") { $0 += "<hint content=\"\($1)\" />" }
        #let mv = "<data class=\"move\" x=\"\(move.x)\" y=\"\(move.y)\" direction=\"\(move.direction)\">\(hints)</data>"
        #self.socket.send(message: "<room roomId=\"\(self.roomId!)\">\(mv)</room>")
        hintsXML = ''
        self.send('<room roomId="%s"><data class="move" x="%d" y="%d" direction="%s">%s</data></room>' % (roomId, posX, posY, directionString, hintsXML))
        
    

In [5]:
import ipywidgets as widgets
from IPython.display import display

client = None
def toggleState(value):
    global client
    
    if value.new:
        client = Client(HOST, PORT, GameLogic())
        client.start()
        value.owner.description = 'Stop Client'
    else:
        client.stop()
        value.owner.description = 'Start Client'
        
toggle = widgets.ToggleButton(description='Start Client', value=False)
#server_toggle.value = True
toggle.observe(toggleState, names='value')

display(toggle)

ToggleButton(value=False, description='Start Client')

In [6]:
client.sock.send('<protocol><join gameType="swc_2019_piranhas"/>'.encode('utf8'))
#dataFromServer = receive()
#roomId = dataFromServer.split('"')[1]
#print(dataFromServer)

AttributeError: 'NoneType' object has no attribute 'sock'

In [ ]:
print(receive())

In [ ]:
move(roomId, 0, 2, 'DOWN_RIGHT')

In [ ]:
import declxml as xml
joined_processor = xml.string('joined', attribute='roomId')
room_processor = xml.dictionary('room', [
    xml.string('data', attribute='class'),
    xml.string('data', attribute='color', required=False, default=None),
    xml.dictionary('data', [
        xml.dictionary('state', [
            
        ], required=False)
    ])
])
protocol_processor = xml.dictionary('protocol', [
    joined_processor,
    xml.string('room', attribute='roomId'),
    xml.array(room_processor)
])

root_room_processor = xml.dictionary('root', [
    xml.array(room_processor, alias='rooms')
])

xml.parse_from_string(root_room_processor, '<root>' + '\n'.join(dataFromServer.split('\n')[2:]) + '</root>')

In [ ]:
print('<root>' + '\n'.join(dataFromServer.split('\n')[2:]) + '</root>')

In [ ]:
import re

In [ ]:
re.match(r'<room([\s\S]+?)<\/room>', 'hallo<room arg="hallo">hello\n\t</room>', re.S|re.M)

In [ ]:
for i, line in enumerate(dataFromServer.split('\n')):
    print(i, line)

In [ ]:
re.match(r'<room([\s\S]+?)</room>', dataFromServer, re.S|re.M)

In [ ]:
dataFromServer

In [ ]:
for group in re.match(r'((?:.)*?(<room.+?</room>)(?:.)*?)+', dataFromServer, re.S|re.M).groups():
    print(group)

# This is where it starts taking form

## Structure through Classes and Stuff

For **testing**: A little xml as string stored from a previous connection to the server.

In [7]:
xml_string = """
<protocol>
  <joined roomId="55b6a2a0-db45-4cf4-9032-e384af7551bb"/>
  <room roomId="55b6a2a0-db45-4cf4-9032-e384af7551bb">
    <data class="welcomeMessage" color="red"/>
  </room>
  <room roomId="55b6a2a0-db45-4cf4-9032-e384af7551bb">
    <data class="memento">
      <state class="sc.plugin2019.GameState" startPlayerColor="RED" currentPlayerColor="RED" turn="0">
        <red displayName="Unknown" color="RED"/>
        <blue displayName="Unknown" color="BLUE"/>
        <board>
          <fields>
            <field x="0" y="0" state="EMPTY"/>
            <field x="0" y="1" state="RED"/>
            <field x="0" y="2" state="RED"/>
            <field x="0" y="3" state="RED"/>
            <field x="0" y="4" state="RED"/>
            <field x="0" y="5" state="RED"/>
            <field x="0" y="6" state="RED"/>
            <field x="0" y="7" state="RED"/>
            <field x="0" y="8" state="RED"/>
            <field x="0" y="9" state="EMPTY"/>
          </fields>
          <fields>
            <field x="1" y="0" state="BLUE"/>
            <field x="1" y="1" state="EMPTY"/>
            <field x="1" y="2" state="EMPTY"/>
            <field x="1" y="3" state="EMPTY"/>
            <field x="1" y="4" state="EMPTY"/>
            <field x="1" y="5" state="EMPTY"/>
            <field x="1" y="6" state="EMPTY"/>
            <field x="1" y="7" state="EMPTY"/>
            <field x="1" y="8" state="EMPTY"/>
            <field x="1" y="9" state="BLUE"/>
          </fields>
          <fields>
            <field x="2" y="0" state="BLUE"/>
            <field x="2" y="1" state="EMPTY"/>
            <field x="2" y="2" state="EMPTY"/>
            <field x="2" y="3" state="EMPTY"/>
            <field x="2" y="4" state="EMPTY"/>
            <field x="2" y="5" state="EMPTY"/>
            <field x="2" y="6" state="EMPTY"/>
            <field x="2" y="7" state="EMPTY"/>
            <field x="2" y="8" state="EMPTY"/>
            <field x="2" y="9" state="BLUE"/>
          </fields>
          <fields>
            <field x="3" y="0" state="BLUE"/>
            <field x="3" y="1" state="EMPTY"/>
            <field x="3" y="2" state="OBSTRUCTED"/>
            <field x="3" y="3" state="EMPTY"/>
            <field x="3" y="4" state="EMPTY"/>
            <field x="3" y="5" state="EMPTY"/>
            <field x="3" y="6" state="EMPTY"/>
            <field x="3" y="7" state="EMPTY"/>
            <field x="3" y="8" state="EMPTY"/>
            <field x="3" y="9" state="BLUE"/>
          </fields>
          <fields>
            <field x="4" y="0" state="BLUE"/>
            <field x="4" y="1" state="EMPTY"/>
            <field x="4" y="2" state="EMPTY"/>
            <field x="4" y="3" state="EMPTY"/>
            <field x="4" y="4" state="EMPTY"/>
            <field x="4" y="5" state="EMPTY"/>
            <field x="4" y="6" state="OBSTRUCTED"/>
            <field x="4" y="7" state="EMPTY"/>
            <field x="4" y="8" state="EMPTY"/>
            <field x="4" y="9" state="BLUE"/>
          </fields>
          <fields>
            <field x="5" y="0" state="BLUE"/>
            <field x="5" y="1" state="EMPTY"/>
            <field x="5" y="2" state="EMPTY"/>
            <field x="5" y="3" state="EMPTY"/>
            <field x="5" y="4" state="EMPTY"/>
            <field x="5" y="5" state="EMPTY"/>
            <field x="5" y="6" state="EMPTY"/>
            <field x="5" y="7" state="EMPTY"/>
            <field x="5" y="8" state="EMPTY"/>
            <field x="5" y="9" state="BLUE"/>
          </fields>
          <fields>
            <field x="6" y="0" state="BLUE"/>
            <field x="6" y="1" state="EMPTY"/>
            <field x="6" y="2" state="EMPTY"/>
            <field x="6" y="3" state="EMPTY"/>
            <field x="6" y="4" state="EMPTY"/>
            <field x="6" y="5" state="EMPTY"/>
            <field x="6" y="6" state="EMPTY"/>
            <field x="6" y="7" state="EMPTY"/>
            <field x="6" y="8" state="EMPTY"/>
            <field x="6" y="9" state="BLUE"/>
          </fields>
          <fields>
            <field x="7" y="0" state="BLUE"/>
            <field x="7" y="1" state="EMPTY"/>
            <field x="7" y="2" state="EMPTY"/>
            <field x="7" y="3" state="EMPTY"/>
            <field x="7" y="4" state="EMPTY"/>
            <field x="7" y="5" state="EMPTY"/>
            <field x="7" y="6" state="EMPTY"/>
            <field x="7" y="7" state="EMPTY"/>
            <field x="7" y="8" state="EMPTY"/>
            <field x="7" y="9" state="BLUE"/>
          </fields>
          <fields>
            <field x="8" y="0" state="BLUE"/>
            <field x="8" y="1" state="EMPTY"/>
            <field x="8" y="2" state="EMPTY"/>
            <field x="8" y="3" state="EMPTY"/>
            <field x="8" y="4" state="EMPTY"/>
            <field x="8" y="5" state="EMPTY"/>
            <field x="8" y="6" state="EMPTY"/>
            <field x="8" y="7" state="EMPTY"/>
            <field x="8" y="8" state="EMPTY"/>
            <field x="8" y="9" state="BLUE"/>
          </fields>
          <fields>
            <field x="9" y="0" state="EMPTY"/>
            <field x="9" y="1" state="RED"/>
            <field x="9" y="2" state="RED"/>
            <field x="9" y="3" state="RED"/>
            <field x="9" y="4" state="RED"/>
            <field x="9" y="5" state="RED"/>
            <field x="9" y="6" state="RED"/>
            <field x="9" y="7" state="RED"/>
            <field x="9" y="8" state="RED"/>
            <field x="9" y="9" state="EMPTY"/>
          </fields>
        </board>
      </state>
    </data>
  </room>
  <room roomId="55b6a2a0-db45-4cf4-9032-e384af7551bb">
    <data class="sc.framework.plugins.protocol.MoveRequest"/>
  </room>
"""

In [83]:
import numpy as np
import bs4
from bs4 import BeautifulSoup

In [84]:
soup = BeautifulSoup(xml_string, 'xml')
print(soup.prettify())

<?xml version="1.0" encoding="utf-8"?>
<protocol>
 <joined roomId="55b6a2a0-db45-4cf4-9032-e384af7551bb"/>
 <room roomId="55b6a2a0-db45-4cf4-9032-e384af7551bb">
  <data class="welcomeMessage" color="red"/>
 </room>
 <room roomId="55b6a2a0-db45-4cf4-9032-e384af7551bb">
  <data class="memento">
   <state class="sc.plugin2019.GameState" currentPlayerColor="RED" startPlayerColor="RED" turn="0">
    <red color="RED" displayName="Unknown"/>
    <blue color="BLUE" displayName="Unknown"/>
    <board>
     <fields>
      <field state="EMPTY" x="0" y="0"/>
      <field state="RED" x="0" y="1"/>
      <field state="RED" x="0" y="2"/>
      <field state="RED" x="0" y="3"/>
      <field state="RED" x="0" y="4"/>
      <field state="RED" x="0" y="5"/>
      <field state="RED" x="0" y="6"/>
      <field state="RED" x="0" y="7"/>
      <field state="RED" x="0" y="8"/>
      <field state="EMPTY" x="0" y="9"/>
     </fields>
     <fields>
      <field state="BLUE" x="1" y="0"/>
      <field state="EMPTY

### GameState Class - Does the Parsing and holds the current State

In [173]:
class GameState():

    def __init__(self, xml_string=None):
        self.roomId = None
        self.ourColor = None
        
        self.currentPlayerColor = None
        self.startPlayerColor = None
        self.turn = None
        self.board = None
        
        if type(xml_string) is str:
            self.parse(xml_string)
    
    @classmethod
    def copy(cls, otherState):
        state = cls()
        state.roomId = otherState.roomId
        state.ourColor = otherState.ourColor
        state.currentPlayerColor = otherState.currentPlayerColor
        state.startPlayerColor = otherState.startPlayerColor
        state.turn = otherState.turn
        if otherState.board is not None:
            state.board = otherState.board.copy()
        
        return state
    
    def parse(self, xml_string):
        if type(xml_string) is not str:
            return False
        
        soup = BeautifulSoup(xml_string, 'xml')
        
        self.parseRoomId(soup)
        
        for roomElem in soup.find_all('room', roomId=roomId):

            cls = roomElem.data.get('class')

            #################################################
            if cls == 'welcomeMessage':
                # the server tells us which color we are
                self.parseWelcomeMessage(roomElem.data)

            #################################################
            elif cls == 'memento':
                # the server tells us the current game state
                self.parseState(roomElem.data)

            #################################################
            elif cls == 'sc.framework.plugins.protocol.MoveRequest':
                print('Move!')

        return True
                
    def parseRoomId(self, soup):
        if soup is None:
            return False
        
        changed = False
        
        joinedTag = soup.find('joined')
        if joinedTag is not None:
            roomId = joinedTag.get('roomId')
            if roomId is not None:
                changed = roomId != self.roomId
                self.roomId = roomId
        
        return changed
    
    
    def parseWelcomeMessage(self, data):
        if data is None:
            return False
        
        changed = False
        
        color = data.get('color')
        if color is not None:
            changed = color != self.ourColor
            self.ourColor = color
        
        return changed
    
    
    def parseState(self, data):
        changed = False
        
        startPlayerColor = data.state.get('startPlayerColor')
        if startPlayerColor is not None:
            startPlayerColor = startPlayerColor.lower()
            changed = changed or startPlayerColor != self.startPlayerColor
            self.startPlayerColor = startPlayerColor
        
        currentPlayerColor = data.state.get('currentPlayerColor')
        if currentPlayerColor is not None:
            currentPlayerColor = currentPlayerColor.lower()
            changed = changed or currentPlayerColor != self.currentPlayerColor
            self.currentPlayerColor = currentPlayerColor
            
        turn = data.state.get('turn')
        if turn is not None:
            try:
                turn = int(turn)
                changed = changed or turn != self.turn
                self.turn = turn
            except:
                pass

        boardChanged = self.parseBoard(data.board)

        return changed or boardChanged
        
    
    def parseBoard(self, boardTag):
        if boardTag is None:
            return False
        
        changed = False
        
        board = np.zeros((10, 10), dtype=np.int)

        for field in boardTag.find_all('field'):
            x = field.get('x')
            y = field.get('y')
            state = field.get('state')

            if x is not None and y is not None:
                try:
                    x = int(x)
                    y = int(y)
                except Exception as ex:
                    print(ex)
                    continue
                
                # TODO: Check x and y for valid range
                if state == 'EMPTY':
                    board[x, y] = 0
                elif state == 'RED':
                    board[x, y] = 1 # TODO: this should always be the opponent
                elif state == 'BLUE':
                    board[x, y] = 2 # TODO: this should always be us
                elif state == 'OBSTRUCTED':
                    board[x, y] = 3
            else:
                print(repr(x), repr(y), repr(state))

        
        if (board != 0).any():
            changed = board != self.board
            self.board = board
        
        return changed
        
    def __str__(self):
        stringRepresentation = 'GameState(\n' + ' ' * 4
        stringRepresentation += 'roomId: ' + str(self.roomId) + '\n' + ' ' * 4
        stringRepresentation += 'ourColor: ' + str(self.ourColor) + '\n' + ' ' * 4
        stringRepresentation += 'currentPlayerColor: ' + str(self.currentPlayerColor) + '\n' + ' ' * 4
        stringRepresentation += 'startPlayerColor: ' + str(self.startPlayerColor) + '\n' + ' ' * 4
        stringRepresentation += 'turn: ' + str(self.turn) + '\n' + ' ' * 4
        
        stringRepresentation += 'board:'
        if self.board is None:
            stringRepresentation += ' ' + str(None) + '\n'
        else:
            for row in self.board:
                stringRepresentation += '\n' + ' ' * 8
                for item in row:
                    stringRepresentation += str(item).rjust(2)
            stringRepresentation += '\n'
            
        stringRepresentation += ')'
        return stringRepresentation

In [183]:
import time
startTime = time.time()
state = GameState(xml_string)
timeTaken = time.time() - startTime
print(state)
print('parsing took me %.2fms' % (timeTaken * 1000))

Move!
GameState(
    roomId: 55b6a2a0-db45-4cf4-9032-e384af7551bb
    ourColor: red
    currentPlayerColor: red
    startPlayerColor: red
    turn: 0
    board:
         0 1 1 1 1 1 1 1 1 0
         2 0 0 0 0 0 0 0 0 2
         2 0 0 0 0 0 0 0 0 2
         2 0 3 0 0 0 0 0 0 2
         2 0 0 0 0 0 3 0 0 2
         2 0 0 0 0 0 0 0 0 2
         2 0 0 0 0 0 0 0 0 2
         2 0 0 0 0 0 0 0 0 2
         2 0 0 0 0 0 0 0 0 2
         0 1 1 1 1 1 1 1 1 0
)
parsing took me 8.66ms


In [184]:
newState = GameState.copy(state)
newState.board[5,5] = -8
print('oldState =', state)
print('newState =', newState)

oldState = GameState(
    roomId: 55b6a2a0-db45-4cf4-9032-e384af7551bb
    ourColor: red
    currentPlayerColor: red
    startPlayerColor: red
    turn: 0
    board:
         0 1 1 1 1 1 1 1 1 0
         2 0 0 0 0 0 0 0 0 2
         2 0 0 0 0 0 0 0 0 2
         2 0 3 0 0 0 0 0 0 2
         2 0 0 0 0 0 3 0 0 2
         2 0 0 0 0 0 0 0 0 2
         2 0 0 0 0 0 0 0 0 2
         2 0 0 0 0 0 0 0 0 2
         2 0 0 0 0 0 0 0 0 2
         0 1 1 1 1 1 1 1 1 0
)
newState = GameState(
    roomId: 55b6a2a0-db45-4cf4-9032-e384af7551bb
    ourColor: red
    currentPlayerColor: red
    startPlayerColor: red
    turn: 0
    board:
         0 1 1 1 1 1 1 1 1 0
         2 0 0 0 0 0 0 0 0 2
         2 0 0 0 0 0 0 0 0 2
         2 0 3 0 0 0 0 0 0 2
         2 0 0 0 0 0 3 0 0 2
         2 0 0 0 0-8 0 0 0 2
         2 0 0 0 0 0 0 0 0 2
         2 0 0 0 0 0 0 0 0 2
         2 0 0 0 0 0 0 0 0 2
         0 1 1 1 1 1 1 1 1 0
)
